<a href="https://colab.research.google.com/github/iodoform/g-stable-diffusion/blob/main/Stable_Diffusion_Helper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI無限ガチャ

画像生成AI [StableDiffusion](https://github.com/CompVis/stable-diffusion)を使って無限に回せるガチャを作りました。

AI画像生成部分は[@fladdict](https://twitter.com/fladdict)氏のカスタム版[stable-diffusion](https://github.com/fladdict/stable-diffusion)からフォークさせていただきました。

下記の**重要**、**利用前の注意**、**お願い**、**謝辞**は@fladdict氏が書いたものをそのまま残してあります。

ご利用の際は順守をお願いします。

## 使い方

* このページ上部のメニューで、「ランタイム > ランタイムのタイプを変更」からGPUを有効化
* [HuggingFace](https://huggingface.co/)でアカウントを作成
* [StableDiffusionのモデルページ](https://huggingface.co/CompVis/stable-diffusion-v-1-4-original)で、「利用規約」に合意する。
* モデルファイル [sd-v1-4.ckpt](https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt) をダウンロード
* モデルファイルを Google Drive等にアップロード
* 下のセル 「1-1. Google Driveとの接続」を実行
* 下のセル　「1-2. のフォーム」に、Google Driveにアップしたモデルのパスをセット
* このページ上部のメニューで、「ランタイム > 全てのセルを実行」を選択

## 不安程な場合
* CUDA Errorが出る場合、メモリが足りてないのでGoogle Colab Pro（or Pro+)を検討ください。

## 重要

「[Stable Diffusionの利用ライセンス](https://huggingface.co/spaces/CompVis/stable-diffusion-license)」を遵守してご利用ください。

----

## 利用前の注意
画像生成AIは、インターネットそのものの縮図です。あらゆるものを生成するので、生成者は自分の生成物に責任をもつ必要があります。

多くの場合、問題ある画像は偶発的というよりは、「生成者が意図的に指示」をすることで生成されます。以下のようなことを心がけましょう。


* ポルノを含む、性的な画像を生成しない（海外基準で罰せられる可能性があります）。
* 攻撃的な画像、差別的な画像、人を不快にする目的の画像を生成しない。
* 政治的な主張に用いない。
* 各種の文化バイアスがかかる場合があります。生成者が適宜バランスを調整をする（例、「結婚式」の画像は欧米式で異性愛の画像になりやすい。医者の画像は白人男性になりやすい、他人の著作物をアップロードしない）。
* 他者の権利を侵害しない（孫悟空やダースベイダーなどを意図的に作らない）。
* 実材の人物、事件、イベントの画像（フェイクニュース含む）を作成しない。
* 現役の作家の画風を単独指名で入力しない（個人的に推奨のマナーです）。

----

## お願い
AIによる画像生成、仕事がなくなるといった文脈で煽る方向の流れは、望むものではありません。
むしろ、みんなで「新しい創作」はどういうものか？アーティストはどうAIを使いこなしていけばいいのか？を模索していければお思います。 

活版印刷が著作権の概念を生み、写真が印象派や抽象芸術の扉を開いたように、新しいテクノロジーは、新しい表現をもたらします。今、必要なことは、みんなであらゆる方向から実験をして、新しい可能性の総当たり探索をすることだと思います。

そんな方向性で使ってもらえればと。


## 謝辞
構成コードは以下の方々のライブラリ、スニペット、コードを参考、あるいは依拠しています。
またnotebookは下記コード群のライセンスを継承します。

* [StableDiffusion](https://github.com/CompVis/stable-diffusion) - [CreativeML Open RAIL-M License](https://github.com/CompVis/stable-diffusion/blob/main/LICENSE)
* [Diffusers](https://github.com/huggingface/diffusers) - [Apache License 2.0](https://github.com/huggingface/diffusers/blob/main/LICENSE)
* KLMSサンプリングは、[@RiversHaveWings](https://twitter.com/RiversHaveWings) 氏の [KLMS Sampling](https://github.com/crowsonkb/k-diffusion.git)より。 [MIT License](https://github.com/crowsonkb/k-diffusion/blob/master/LICENSE)
* プロンプトのウェイト処理は、[@Lincoln Stein](https://github.com/lstein)氏のカスタム版[StableDiffusion](https://github.com/lstein/stable-diffusion)より。 [MIT LICENSE](https://github.com/lstein/stable-diffusion/blob/main/LICENSE)
* KLMS連携の理解に [@pharmapsychotic](https://twitter.com/pharmapsychotic)氏の[Stable Diffusion notebook](https://colab.research.google.com/github/pharmapsychotic/ai-notebooks/blob/main/pharmapsychotic_Stable_Diffusion.ipynb#scrollTo=UU52ZvES6-1T)。

# セットアップ

In [ ]:
#@markdown ## 1-1. Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown ## 1-2. Google Driveにアップしたモデルのパスを設定
#@markdown 左メニューからGoogle Driveを掘り、アップロードしたckptファイルを選択し、右クリックから「パスをコピー」を行います。
#@markdown コピーしたパスを下のフォームにコピペしてください。

GDRIVE_MODEL_PATH = "/content/drive/MyDrive/stableDiffusion/sd-v1-4.ckpt" #@param {type:"string"}

In [ ]:
#@markdown ## 1-3. 画像の保存設定
DRIVE_PATH = "/content/drive/MyDrive" #Driveのルート
SAVE_FILE = False #@param {type:"boolean"}
SAVE_FILE_PATH = "/content/drive/MyDrive/stable-diffusion/output" #@param {type:"string"}
SAVE_FILE_PREFIX = "SD" #@param {type:"string"}

In [ ]:
#GPUの確認
!nvidia-smi -L

In [ ]:
#必要ファイルのインストール
%cd /content/

#GIT
!git clone https://github.com/CompVis/stable-diffusion  
%cd /content/stable-diffusion

!git clone https://github.com/CompVis/taming-transformers
!git clone https://github.com/openai/CLIP.git
!git clone https://github.com/crowsonkb/k-diffusion.git
!git clone https://github.com/iodoform/g-stable-diffusion.git

#PIP
!pip install albumentations 
!pip install diffusers==0.2.4 
!pip install gradio 
!pip install numpy einops kornia
!pip install omegaconf
!pip install pytorch-lightning
!pip install torch-fidelity
!pip install transformers
!pip install ftfy jsonmerge resize-right torchdiffeq tqdm

#Pathを通す
import sys
sys.path.append(".")
sys.path.append("./CLIP")
sys.path.append('./taming-transformers')
sys.path.append('./k-diffusion')
sys.path.append('./ldm')

#k_diffusionは初期化が必要
!echo '' > ./k-diffusion/k_diffusion/__init__.py

In [ ]:
import argparse, gc, json, os, random, sys, time, glob, requests
import torch
import torch.nn as nn
import numpy as np
import PIL
from contextlib import contextmanager, nullcontext
from einops import rearrange, repeat
from IPython.display import display, clear_output
from itertools import islice
from omegaconf import OmegaConf
from PIL import Image
from pytorch_lightning import seed_everything
from torch.cuda.amp import autocast
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler

from k_diffusion.sampling import sample_lms
from k_diffusion.external import CompVisDenoiser

from diffusers.pipelines.stable_diffusion.safety_checker import StableDiffusionSafetyChecker
from transformers import AutoFeatureExtractor

from datetime import datetime
import gc

#メモリのクリーンアップ
def clear_memory():
  gc.collect()
  torch.cuda.empty_cache() 


#新しいDenoiser
class CFGDenoiser(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.inner_model = model

    def forward(self, x, sigma, uncond, cond, cond_scale):
        x_in = torch.cat([x] * 2)
        sigma_in = torch.cat([sigma] * 2)
        cond_in = torch.cat([uncond, cond])
        uncond, cond = self.inner_model(x_in, sigma_in, cond=cond_in).chunk(2)
        return uncond + (cond - uncond) * cond_scale


#Config用クラス
class SDOption():
  def __init__(self):
    self.ckpt = GDRIVE_MODEL_PATH
    self.config = 'configs/stable-diffusion/v1-inference.yaml'
    self.ddim_eta = 0.0
    self.ddim_steps = 50
    self.embedding = None # TextInversion対応用のEmbedding.pyファイルへのパス
    self.fixed_code = True
    self.init_img = None
    self.init_mask = None
    self.n_iter = 1
    self.n_samples = 1
    self.outdir = SAVE_FILE_PATH
    self.precision = 'full' # 'autocast'
    self.prompt = ""
    self.sampler = 'klms'
    self.save = True
    self.scale = 7.5
    self.seed = -1
    self.strength = 0.5
    self.variations_mode = True #Export variations of init Image
    self.H = 512
    self.W = 512
    self.C = 4
    self.f = 8
    self.prompt_conditioning = None #promptをテキストでなくconditioningで直接注入する場合


#第3パラメーターにEmbeddingPath（Text InversionでトレーニングしたTextEmbedderを指定可能に）
class SDHelper():
  def __init__(self, config_path, model_path, embedding_path = None):
    config = OmegaConf.load(config_path)
    self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    self.model = self.load_model_from_config(config, model_path).to(self.device)
    self.safety_model_id = "CompVis/stable-diffusion-safety-checker"
    self.safety_feature_extractor = AutoFeatureExtractor.from_pretrained(self.safety_model_id)
    self.safety_checker = StableDiffusionSafetyChecker.from_pretrained(self.safety_model_id)

    if embedding_path != None:
      if hasattr(self.model, "embedding_manager"):
        self.model.embedding_manager.load(embedding_path)
      else:
        print("Warning: Model does not have embedding_manager")


  def chunk(self, it, size):
      it = iter(it)
      return iter(lambda: tuple(islice(it, size)), ())


  def load_model_from_config(self, config, ckpt, verbose=False):
      print(f"Loading model from {ckpt}")
      pl_sd = torch.load(ckpt, map_location="cpu")
      if "global_step" in pl_sd:
          print(f"Global Step: {pl_sd['global_step']}")
      sd = pl_sd["state_dict"]
      model = instantiate_from_config(config.model)
      m, u = model.load_state_dict(sd, strict=False)
      if len(m) > 0 and verbose:
          print("missing keys:")
          print(m)
      if len(u) > 0 and verbose:
          print("unexpected keys:")
          print(u)

      model.cuda()
      model.eval()
      return model

  def make_batch(self, image, mask):
      image = np.array(Image.open(image).convert("RGB"))
      image = image.astype(np.float32)/255.0
      image = image[None].transpose(0,3,1,2)
      image = torch.from_numpy(image)

      mask = np.array(Image.open(mask).convert("L"))
      mask = mask.astype(np.float32)/255.0
      mask = mask[None,None]
      mask[mask < 0.5] = 0
      mask[mask >= 0.5] = 1
      mask = torch.from_numpy(mask)

      masked_image = (1-mask)*image

      batch = {"image": image, "mask": mask, "masked_image": masked_image}
      for k in batch:
          batch[k] = batch[k].to(device=self.device)
          batch[k] = batch[k]*2.0-1.0
      return batch


  def load_img(self, path, w, h):
    if path.startswith('http://') or path.startswith('https://'):
        image = Image.open(requests.get(path, stream=True).raw).convert('RGB')
    else:
        if os.path.isdir(path):
            files = [file for file in os.listdir(path) if file.endswith('.png') or file .endswith('.jpg')]
            path = os.path.join(path, random.choice(files))
            print(f"Chose random init image {path}")
        image = Image.open(path).convert('RGB')
    image = image.resize((w, h), Image.LANCZOS)
    w, h = image.size
    w, h = map(lambda x: x - x % 32, (w, h))  # resize to integer multiple of 32
    image = image.resize((w, h), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.


  def numpy_to_pil(self, images):
    if images.ndim == 3:
        images = images[None, ...]
    images = (images * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    return pil_images


  def load_replacement(self, x):
      try:
          hwc = x.shape
          #セーフフィルターのリック・ストレイは、日本では法律に触れる可能性があるので、違う画像に差し替えます。
          #y = Image.open("assets/rick.jpeg").convert("RGB").resize((hwc[1], hwc[0]))
          y = PIL.Image.new(mode="RGB", size=(hwc[1], hwc[0]))
          y = (np.array(y)/255.0).astype(x.dtype)
          assert y.shape == x.shape
          return y
      except Exception:
          return x


  def check_safety(self, x_image):
    safety_checker_input = self.safety_feature_extractor(self.numpy_to_pil(x_image), return_tensors="pt")
    x_checked_image, has_nsfw_concept = self.safety_checker(images=x_image, clip_input=safety_checker_input.pixel_values)
    assert x_checked_image.shape[0] == len(has_nsfw_concept)
    for i in range(len(has_nsfw_concept)):
        if has_nsfw_concept[i]:
            x_checked_image[i] = self.load_replacement(x_checked_image[i])
    return x_checked_image, has_nsfw_concept


  def get_prompt_weight(self, prompt):
    return self.model.get_learned_conditioning(prompt)
    

  def generate(self, opt):
      global sample_idx
      seed_everything(opt.seed)

      #出力ディレクトリの作成
      os.makedirs(opt.outdir, exist_ok=True)

      #サンプラー選択
      if opt.sampler == 'plms':
          sampler = PLMSSampler(self.model)
      else:
          sampler = DDIMSampler(self.model)

      model_wrap = CompVisDenoiser(self.model)       
      batch_size = opt.n_samples

      #promptをバッチの数だけコピー
      prompt = opt.prompt
      assert prompt is not None
      data = [batch_size * [prompt]]


      #初期画像の潜在空間を作成
      init_latent = None
      if opt.init_img != None and opt.init_img != '':
          init_image = self.load_img(opt.init_img, opt.W, opt.H).to(self.device)
          init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
          init_latent = self.model.get_first_stage_encoding(self.model.encode_first_stage(init_image))  # move to latent space


      #Inpaint用（つくりかけ）
      """
      if opt.init_mask != None:
        init_image = self.load_img(opt.init_img, opt.W, opt.H).to(self.device)
        init_mask = self.load_img(opt.init_mask, opt.W, opt.H).to(self.device)
        init_masked_image = (1-init_mask)*init_image
        batch = {"image": init_image, "mask": init_mask, "masked_image": init_masked_image}"""

      sampler.make_schedule(ddim_num_steps=opt.ddim_steps, ddim_eta=opt.ddim_eta, verbose=False)
      t_enc = int(opt.strength * opt.ddim_steps)


      #?? txt2imgで使われる初期値っぽいが…？
      start_code = None
      if opt.fixed_code and init_latent == None:
          start_code = torch.randn([opt.n_samples, opt.C, opt.H // opt.f, opt.W // opt.f], device=self.device)

      precision_scope = autocast if opt.precision == "autocast" else nullcontext

      images = []
      with torch.no_grad():
          with precision_scope("cuda"):
              with self.model.ema_scope():
                  for n in range(opt.n_iter):
                      for prompts in data:
                          #init_latentに初期画像の潜在空間
                          #cにコンディショナル条件の潜在空間
                          #ucにあんコンディショナルの潜在

                          uc = None
                          if opt.scale != 1.0:
                              uc = self.model.get_learned_conditioning(batch_size * [""])

                          if isinstance(prompts, tuple):
                              prompts = list(prompts)

                          #プロンプトのウェイト処理
                          #全てのプロンプトに正規化したウェイトをかけて合算する
                          subprompts, weights = SDHelper.prompt_splitter(prompts[0])
                          if len(subprompts) > 1:
                            c = torch.zeros_like(uc)
                            # get total weight for normalizing
                            totalWeight = sum(weights)
                            # normalize each "sub prompt" and add it
                            for i in range(0,len(subprompts)):
                              weight = weights[i]
                              #if not skip_normalize:
                              # skip_normalizeがついてる意図が不明なので外す。
                              weight = weight / totalWeight
                              c = torch.add(c,self.model.get_learned_conditioning(subprompts[i]), alpha=weight)
                          else: # just standard prompt
                            c = self.model.get_learned_conditioning(prompts)

                          
                          #promptをテキストでなく Tensorで直接注入する場合
                          if (opt.prompt_conditioning != None):
                            c = opt.prompt_conditioning


                          if init_latent != None:
                              #Img2Ima
                              
                              #z_enc に 初期画像の潜在空間
                              #cにテキストの潜在空間
                              #t_encに 画像のstrength * ddimsteps?
                              #ucに空白
                              z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(self.device))
                              samples = sampler.decode(z_enc, 
                                                       c, 
                                                       t_enc, 
                                                       unconditional_guidance_scale=opt.scale,
                                                       unconditional_conditioning=uc)
                          else:
                              if opt.sampler == 'klms':
                                  print("Using KLMS sampling")
                                  shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                                  sigmas = model_wrap.get_sigmas(opt.ddim_steps)
                                  model_wrap_cfg = CFGDenoiser(model_wrap)
                                  x = torch.randn([opt.n_samples, *shape], device=self.device) * sigmas[0]
                                  extra_args = {'cond': c, 'uncond': uc, 'cond_scale': opt.scale}
                                  samples = sample_lms(model_wrap_cfg, 
                                                       x, 
                                                       sigmas, 
                                                       extra_args=extra_args, 
                                                       disable=False)
                              else:
                                  shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                                  samples, _ = sampler.sample(S=opt.ddim_steps,
                                                                  conditioning=c,
                                                                  batch_size=opt.n_samples,
                                                                  shape=shape,
                                                                  verbose=False,
                                                                  unconditional_guidance_scale=opt.scale,
                                                                  unconditional_conditioning=uc,
                                                                  eta=opt.ddim_eta,
                                                                  x_T=start_code)

                          x_samples = self.model.decode_first_stage(samples)
                          x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)
                          x_samples = x_samples.cpu().permute(0, 2, 3, 1).numpy()

                          #Safety Checker added
                          x_checked_image, has_nsfw_concept = self.check_safety(x_samples)
                          x_checked_image_torch = torch.from_numpy(x_checked_image).permute(0, 3, 1, 2)

                          for x_sample in x_checked_image_torch:
                            x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                            images.append(Image.fromarray(x_sample.astype(np.uint8)))
                            if (opt.save==True):
                              file_id = datetime.today().strftime('%Y-%m-%d-%H-%M-%S')
                              filepath = os.path.join(opt.outdir, f"{SAVE_FILE_PREFIX}-{file_id}.png")
                              Image.fromarray(x_sample.astype(np.uint8)).save(filepath)
                              #sample_idx += 1
      return images

  #Prompt Splitter is based on Lincoln Stein's code
  #https://github.com/lstein/stable-diffusion/blob/main/ldm/simplet2i.py

  #MidJourney互換でプロンプトの重さを処理するコード
  #任意の文字列、 :: （スペース入るかも）（数字はいるかも）　（スペース入るかも）
  def prompt_splitter(text):
    """
    grabs all text up to the first occurrence of ':' 
    uses the grabbed text as a sub-prompt, and takes the value following ':' as weight
    if ':' has no value defined, defaults to 1.0
    repeats until no text remaining
    """
    remaining = len(text)
    prompts = []
    weights = []
    while remaining > 0:
        if "::" in text:
            idx = text.index("::") # first occurrence from start
            # grab up to index as sub-prompt
            prompt = text[:idx]
            remaining -= idx
            # remove from main text
            text = text[idx+2:]
            # find value for weight 
            if " " in text:
                idx = text.index(" ") # first occurence
            else: # no space, read to end
                idx = len(text)
            if idx != 0:
                try:
                    weight = float(text[:idx])
                except: # couldn't treat as float
                    print(f"Warning: '{text[:idx]}' is not a value, are you missing a space?")
                    weight = 1.0
            else: # no value found
                weight = 1.0
            # remove from main text
            remaining -= idx
            text = text[idx+1:]
            # append the sub-prompt and its weight
            prompts.append(prompt)
            weights.append(weight)
        else: # no : found
            if len(text) > 0: # there is still text though
                # take remainder as weight 1
                prompts.append(text)
                weights.append(1.0)
            remaining = 0
    print(prompts)
    print(weights)
    return prompts, weights


#SDHelperのインスタンス化
#第3引数にEmbeddingの.ptを指定すれば、TextInversionに対応可能
opt = SDOption()
sdh = SDHelper(opt.config, opt.ckpt, "/content/drive/MyDrive/stable-diffusion/embeddings_gs-40000.pt")

# ガチャを起動

#バッチ処理用

* Init Image: ベース画像（オプション）
* Prompt: 生成用のテキスト
* Width: 幅
* Height: 高さ
* Cfg Scale: テキスト誘導の強さ（初期値 7.5)
* Steps: 画像の描き込み時間。多いほど時間がかかり詳細になる。(初期値 50）
* Init Image Strength: ベース画像をどれほど残すか（0: 無視 〜 1: オリジナル画像そのまま）
* Num: 1回に生成する枚数
* Seed: 画像の生成元となる乱数（-1にすると毎回ランダム）

In [ ]:
#ガチャを起動
import random
import torch
import gradio as gr

import re
from PIL import Image, ImageFont, ImageDraw, ImageFilter, ImageOps
from io import BytesIO
import base64
import imageio
import numpy as np

clear_memory()


def diffuse(init_image, prompt, width, height, guidance_scale, steps, init_strength, num, seed):
  result = list()
  """
  if init_image != None:
    imageio.imwrite("data.png", init_image["image"])
    imageio.imwrite("data_mask.png", init_image["mask"]) 
    init_image = Image.open("data.png")
    mask_image = Image.open("data_mask.png")
    #display(init_image)
    #display(mask_image)
  """
    
  opt.init_img = init_image 
  #init_image["image"]
  #opt.init_mask = init_image["mask"]
  opt.strength = 1- init_strength
  opt.prompt = prompt
  opt.W = width
  opt.H = height
  opt.scale = guidance_scale
  opt.ddim_steps = steps
  opt.n_iter = 1
  opt.save == SAVE_FILE
  
  for index in range(int(num)):
    opt.seed = random.randint(0, 2**32) if seed == -1 else seed
    image = sdh.generate(opt)[0]
    #display(image)
    result.append(image)
  return result


def image_clear(image_init, strength_sli):
  print("image clear", image_init)


def image_change(image_init, strength_sli):
  if image_init == None:
    return gr.Slider.update(visible=False)
  return gr.Slider.update(visible=True)


def set_image_to_init(images):
  if len(images)==0:
    return
  try:
    image_data = re.sub('^data:image/.+;base64,', '', images[0])
    image = Image.open(BytesIO(base64.b64decode(image_data)))
    return image
  except IndexError:
    print("failed to get image")
    return
def drawflame(img,thickness,color):
  im = Image.new('RGB', (thickness*2+img.size[0],thickness*2+img.size[1]), color)
  im.paste(img,(thickness,thickness))
  return im

font = ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf', 100)

n_img = Image.open("/content/stable-diffusion/g-stable-diffusion/card_flame/N.png")
ImageDraw.Draw(n_img).text((10, 0), "N", font=font,stroke_width=10,stroke_fill='black')
r_img = Image.open("/content/stable-diffusion/g-stable-diffusion/card_flame/R.png")
ImageDraw.Draw(r_img).text((10, 0), "R", font=font,stroke_width=10,stroke_fill='black')
sr_img = Image.open("/content/stable-diffusion/g-stable-diffusion/card_flame/SR.png")
ImageDraw.Draw(sr_img).text((10, 0), "SR", font=font,stroke_width=10,stroke_fill='black')
ssr_img = Image.open("/content/stable-diffusion/g-stable-diffusion/card_flame/SSR.png")
ImageDraw.Draw(ssr_img).text((10, 0), "SSR", font=font,stroke_width=10,stroke_fill='black')
prompt = "japanese anime of a beaultiful girl,fantasy costume,fantasy background,beautiful composition,cinematic lighting,pixiv,light novel,digital painting,extremely detailed,sharp focus,ray tracing,8k,cinematic postprocessing"

#ガチャを回す

In [ ]:
#@markdown # プロンプト設定
#@markdown プロンプトを変更する場合は、下記のpromptを書き換えて、このセルを実行してください。
prompt = "japanese anime of a beaultiful girl,fantasy costume,fantasy background,beautiful composition,cinematic lighting,pixiv,light novel,digital painting,extremely detailed,sharp focus,ray tracing,8k,cinematic postprocessing" #@param {type:"string"}

In [ ]:
#@markdown # 回す
#@markdown レアリティはN,R,SR,SSRの4種類です。にぎやかしに数字が入っていますが特に意味はありません。
gacha_illustration = diffuse(init_image =None,prompt =prompt, width=512, height=512, guidance_scale=10, steps = 50, init_strength=0, num=1, seed =-1)[0]
rearity_num = random.randint(0, 9)
rearity = ""
imgsize = (630,890)

if rearity_num==0 or rearity_num==1 or rearity_num==2 or rearity_num==3:
  img = n_img
elif rearity_num==4 or rearity_num==5 or rearity_num==6:
  img = r_img
elif rearity_num==7 or rearity_num==8:
  img = sr_img
else:
  img = ssr_img
img.paste(drawflame(gacha_illustration, 10,(0,0,0)),(49,110))
tmpim = Image.new('RGB', (512,100), (256,256,256))
img.paste(drawflame(gacha_illustration, 10,(0,0,0)),(49,110))
img.paste(drawflame(tmpim, 5,(0,0,0)),(54,700))
font = ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf', 50)
ImageDraw.Draw(img).text((290, 725),str(random.randint(0,9)), font=font,stroke_width=10,stroke_fill='black')
drawflame(img,10,(0,0,0))